In [92]:
# Bibliotecas
import re
import unicodedata
import numpy as np
import pandas as pd

In [152]:
def converter_string_numero(x):
    '''
    
    '''
    if type(x) == str:
        xo = x
        # Aplicar substituições padrões
        x = x.replace(',','.').replace('%', '')
        # Garantir que tenhamos um número e nenhuma letra
        if bool(re.search(r'\d', x)) is True and bool(re.search(r'[a-zA-Z]', x)) is False:
            # Remover problema de excesso de pontos
            if len(x.split('.')) > 1:
                x = ''.join(x.split('.')[:-1]) + '.' + x.split('.')[-1]
            # Report
            #print(f'> De {xo} para {x} que é numérico.')
            # Retorno
            return float(x)
        else:
            return np.nan
    else:
        return x

In [153]:
# Carregar dados brutos
df = pd.read_csv('dados/1_EXT/raw_fundamentus_2023-11-02.csv')

In [154]:
# Fazer tratamentos iniciais
df.columns = [c.upper().replace('?','').strip() for c in df.columns] 

In [155]:
# Colunas para derrubar
cols_to_drop = ['EBIT',
                'DEPÓSITOS',
                'OSCILAÇÕES',
                'UNNAMED: 25',
                'REC SERVIÇOS',
                'LUCRO LÍQUIDO',
                'ÚLTIMOS 3 MESES',
                'RECEITA LÍQUIDA',
                'CART. DE CRÉDITO',
                'ÚLTIMOS 12 MESES',
                'RESULT INT FINANC',
                'DADOS BALANÇO PATRIMONIAL',
                'INDICADORES FUNDAMENTALISTAS',
                'DADOS DEMONSTRATIVOS DE RESULTADOS']
df.drop(columns=cols_to_drop, inplace=True)

In [156]:
# Renomear colunas
df.rename(columns={'DIA': 'OSCILACOES_%_DIA',
                   'MES': 'OSCILACOES_%_MES',
                   '30 DIAS': 'OSCILACOES_%_30_DIAS',
                   '12 MESES': 'OSCILACOES_%_12_MESES',
                   '2023': 'OSCILACOES_%_2023',
                   '2022': 'OSCILACOES_%_2022',
                   '2021': 'OSCILACOES_%_2021',
                   '2020': 'OSCILACOES_%_2020',
                   '2019': 'OSCILACOES_%_2019',
                   '2018': 'OSCILACOES_%_2018',
                   '2017': 'OSCILACOES_%_2017',
                   '2016': 'OSCILACOES_%_2016'}, inplace=True)

In [157]:
# Fazer tratamentos adicionais
df.columns = [re.sub('\s+', ' ', c) for c in df.columns]
df.columns = [c.replace('. ', '_') for c in df.columns]
df.columns = [c.replace(' / ', '_POR_') for c in df.columns]
df.columns = [c.replace('/', '_POR_') for c in df.columns]
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('(', '') for c in df.columns]
df.columns = [c.replace(')', '') for c in df.columns]
df.columns = [unicodedata.normalize('NFD', c).encode('ascii', 'ignore').decode("utf-8")  for c in df.columns]
df.columns = [re.sub('_+', '_', c) for c in df.columns]

In [158]:
# Colunas de atributos
cols_atr = ['PAPEL', 'TIPO', 'EMPRESA', 'SETOR', 'SUBSETOR', 'DATA_ULT_COT', 'ULT_BALANCO_PROCESSADO']
cols_num = [c for c in df.columns if c not in cols_atr] 

In [159]:
# Criar backup para comparar
df_bkp = df.copy()

In [160]:
# Converter colunas numericas
for cn in cols_num:
    df[cn] = df[cn].apply(lambda x:converter_string_numero(x))

In [161]:
# Checar tipos
df[cols_num].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 47 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   COTACAO                450 non-null    float64
 1   MIN_52_SEM             450 non-null    float64
 2   MAX_52_SEM             450 non-null    float64
 3   VOL_$_MED_2M           450 non-null    int64  
 4   VALOR_DE_MERCADO       450 non-null    int64  
 5   VALOR_DA_FIRMA         409 non-null    float64
 6   NRO_ACOES              450 non-null    int64  
 7   OSCILACOES_%_DIA       450 non-null    float64
 8   MES                    450 non-null    float64
 9   OSCILACOES_%_30_DIAS   450 non-null    float64
 10  OSCILACOES_%_12_MESES  450 non-null    float64
 11  OSCILACOES_%_2023      447 non-null    float64
 12  OSCILACOES_%_2022      449 non-null    float64
 13  OSCILACOES_%_2021      450 non-null    float64
 14  OSCILACOES_%_2020      450 non-null    float64
 15  OSCILA

In [163]:
# Salvar
df.to_csv('dados/2_TRA/fundamentus_2023-11-02.csv', index=False)